In [1]:
import numpy as np
import pandas as pd
from sodapy import Socrata

In [6]:
client = Socrata("data.cityofnewyork.us", None)
# https://data.cityofnewyork.us/resource/uacg-pexx.json

In [35]:
results = client.get("uacg-pexx", where="tpep_pickup_datetime < '2016-01-15T01:00:00.000'", order="'tpep_pickup_datetime DESC'", limit=20)

In [43]:
results = client.get("uacg-pexx", where="PULocationID = '138'", limit = 10000000)

In [44]:
data = pd.DataFrame.from_records(results)

In [45]:
data.shape

(1529683, 17)

In [47]:
columns = data.columns

In [48]:
columns

Index([u'dolocationid', u'extra', u'fare_amount', u'improvement_surcharge',
       u'mta_tax', u'passenger_count', u'payment_type', u'pulocationid',
       u'ratecodeid', u'store_and_fwd_flag', u'tip_amount', u'tolls_amount',
       u'total_amount', u'tpep_dropoff_datetime', u'tpep_pickup_datetime',
       u'trip_distance', u'vendorid'],
      dtype='object')

In [50]:
#data = data['dolocationid', 'passenger_count', 'pulocationid', 'total_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_distance']

In [51]:
data = data.drop(['extra', 'fare_amount', 'improvement_surcharge', 'mta_tax', 'payment_type', 'ratecodeid', 
                 'store_and_fwd_flag', 'tip_amount', 'tolls_amount', 'vendorid'], axis=1)

In [52]:
data.head()

dolocationid passenger_count pulocationid total_amount  \
0           48               1          138        45.34   
1          231               1          138        53.76   
2          236               1          138        44.84   
3          236               1          138        35.76   
4          264               1          138        16.34   

     tpep_dropoff_datetime     tpep_pickup_datetime trip_distance  
0  2016-10-18T17:14:12.000  2016-10-18T16:30:11.000         11.23  
1  2016-11-15T10:49:58.000  2016-11-15T09:42:00.000         11.82  
2  2016-09-23T14:41:31.000  2016-09-23T13:55:20.000           8.9  
3  2016-12-02T19:14:59.000  2016-12-02T18:44:53.000          8.95  
4  2016-09-20T21:07:03.000  2016-09-20T21:07:03.000             0

In [53]:
data.to_csv('../../raw_data/full_taxi_2016Jul-Dec.csv')

In [54]:
data = data.drop(['dolocationid', 'pulocationid', 'total_amount', 'tpep_dropoff_datetime', 'trip_distance'], axis=1)

In [55]:
data.head()

passenger_count     tpep_pickup_datetime
0               1  2016-10-18T16:30:11.000
1               1  2016-11-15T09:42:00.000
2               1  2016-09-23T13:55:20.000
3               1  2016-12-02T18:44:53.000
4               1  2016-09-20T21:07:03.000

In [56]:
data = data.set_index('tpep_pickup_datetime')

In [57]:
data.index = pd.DatetimeIndex(data.index)

In [62]:
data.passenger_count = data.passenger_count.astype(int)

In [67]:
data['num_pickups'] = 1

In [71]:
data.head()

passenger_count  num_pickups
tpep_pickup_datetime                              
2016-10-18 16:30:11                 1            1
2016-11-15 09:42:00                 1            1
2016-09-23 13:55:20                 1            1
2016-12-02 18:44:53                 1            1
2016-09-20 21:07:03                 1            1

In [73]:
data = data.resample('H').agg({'num_pickups' : 'count', 'passenger_count' : 'sum'})

In [77]:
data = data.rename(columns={'passenger_count':'num_passengers'})

In [78]:
data.head()

num_passengers  num_pickups
tpep_pickup_datetime                             
2016-07-01 00:00:00            529.0          293
2016-07-01 01:00:00             13.0            8
2016-07-01 02:00:00              2.0            1
2016-07-01 03:00:00              1.0            1
2016-07-01 04:00:00              9.0            9

In [81]:
column_order = ['num_pickups', 'num_passengers']
data = data.reindex(columns=column_order)

In [86]:
data.to_csv('../../clean_data/2016Jul-Dec_clean.csv')

In [89]:
dates = pd.date_range(pd.Timestamp('2016-07-01 00:00:00'), pd.Timestamp('2016-12-31 23:00:00'), freq='H')

In [92]:
data = data.reindex(dates)

In [93]:
data.isnull().sum()

num_pickups         0
num_passengers    148
dtype: int64

In [ ]:
data.fillna(num_passengers)